In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tidynamics
import subprocess
import os, os.path
import time
import itertools
from glob import glob

In [ ]:
cd ~/compute/ABP_lammps/probe_campaign_2/

In [ ]:
%run abp_util

In [ ]:
abp_fr_TH = {
    'params': {
    'tau_r': 10,
    'v0': 10,
    'hfile': 'abp.h5',
    'sigma_probe': 3,
    'add_force': 0.2,
    'sampling': 2*32000,
    'dump_every': 100,
    'L': 30,
    'rho': 0.2,
    'bath_dump_every': 0,
    },
    'vary_k': 'add_force',
    'vary_v': [0., 0.1, 0.2, 0.3, 0.4, 0.5],
    'run_name': 'abp_fr_TH'
}

abp_fr_TW = {
    'params': {
    'tau_r': 10,
    'v0': 10,
    'hfile': 'abp.h5',
    'sigma_probe': 3,
    'add_force': 0,
    'sampling': 2*32000,
    'dump_every': 100,
    'L': 30,
    'rho': 0.2,
    'bath_dump_every': 0,
    },
    'vary_k': 'v0',
    'vary_v': [0, 5, 10, 15, 20],
    'run_name': 'abp_fr_TW'
}

abp_fr_FO = {
    'params': {
    'tau_r': 10,
    'v0': 10,
    'hfile': 'abp.h5',
    'sigma_probe': 3,
    'sampling': 2*32000,
    'dump_every': 100,
    'L': 30,
    'rho': 0.2,
    'bath_dump_every': 0,
    },
    'vary_k': 'v0',
    'vary_v': [0, 5, 10, 15, 20],
    'run_name': 'abp_fr_FO'
}


settings = abp_fr_FO

params = settings['params']

In [ ]:
vary_k

In [ ]:
run_name = settings['run_name']
#os.mkdir(run_name)

in_file_list = []
letters = 'abcdef'
for (i, val), letter in itertools.product(enumerate(settings['vary_v']), letters):
    prefix = f'{run_name}_{i:05d}_{letter}'
    p = params.copy()
    p.update({settings['vary_k']: val, 'hfile': prefix+'.h5'})
    if letter == 'a':
        print(p)
    parameter_file = write_input(**p)
    in_file = prefix+'.in'
    in_file_list.append(in_file)
    with open(os.path.join(run_name, in_file), 'w') as f:
        f.write(parameter_file)
        
print(in_file_list)

In [ ]:
lmp_exe = os.path.join(os.path.expanduser('~'), 'code', 'lammps', 'build_fix_active', 'lmp')

t0 = time.time()

subprocess.run(['parallel', '-j', '3', lmp_exe,'-i', ':::', *in_file_list])

print("elapsed time:", time.time()-t0)

In [ ]:
res = {}

for (i, val), letter in itertools.product(enumerate(vary_v), letters):
    prefix = f'{run_name}_{i:05d}_{letter}'
    p = params.copy()
    p.update({vary_k: val, 'hfile': prefix+'.h5'})
    
    with h5py.File(os.path.join(run_name, p['hfile']), 'r') as a:
        L = a['particles/probe/box/edges/value'][0]
        r = a['particles/probe/position/value'][:,0,:]
        im = a['particles/probe/image/value'][:,0,:]
        r += im*L[None,:]
        v = a['particles/probe/velocity/value'][:,0,:]
        r_t = a['particles/probe/position/time'][:]
        v_t = a['particles/probe/velocity/time'][:]

    r_t -= r_t[0]
    v_t -= v_t[0]
    u = v / np.sqrt(np.sum(v**2, axis=1))[:,None]

    d = {}
    d['msd'] = tidynamics.msd(r)
    d['vacf'] = tidynamics.acf(v)
    d['uacf'] = tidynamics.acf(u)
    d['x'] = r[:,0]

    res[prefix] = d

    print(val)


In [ ]:
plt.figure()

for val in vary_v:
    d = res[val]
    plt.plot(r_t, d['msd'], label=vary_k+' '+str(val))

plt.plot(r_t, 4/3*r_t, label='passive diff')

plt.legend()

In [ ]:
plt.figure()

for val in vary_v:
    d = res[val]
    plt.plot(v_t, d['vacf'])

plt.xlim(0, 10)

In [ ]:
plt.figure()

mu_data = []

for (i, val) in enumerate(settings['vary_v'][1:]):
    x = res_x[val]
    force = val
    r_t = r_dt*np.arange(len(x))
    l, = plt.plot(r_t, x/force, label=vary_k+' '+str(val))
    fit = np.polyfit(r_t, x/force, 1)
    plt.plot(r_t, np.poly1d(fit)(r_t), c=l.get_color(), ls='--')
    slope, x0 = fit
    print(str(val), "mobility", slope/force)
    mu_data.append(slope/force)
    

plt.legend()

In [ ]:
plt.figure()
plt.plot(settings['vary_v'][1:], mu_data)

In [ ]:
plt.figure()

D_data = []

for (i, val) in enumerate(settings['vary_v'][1:]):
    msd = res_msd[val]
    r_t = r_dt*np.arange(len(msd))
    l, = plt.plot(r_t, msd, label=settings['vary_k']+' '+str(val))
    fit = np.polyfit(r_t, msd, 1)
    plt.plot(r_t, np.poly1d(fit)(r_t), c=l.get_color(), ls='--')
    slope, x0 = fit
    print(str(val), "diffusion", slope/4)
    D_data.append(slope/4)
    
plt.legend()

In [ ]:
plt.figure()

vsq_data = []
for (i, val) in enumerate(settings['vary_v'][1:]):
    vacf = res_vacf[val]
    r_t = v_dt*np.arange(len(vacf))
    l, = plt.plot(r_t, vacf, label=settings['vary_k']+' '+str(val))
    vsq_data.append(vacf[0])

plt.xlim(0, 5)
plt.legend()

In [ ]:
plt.figure()

probe_mass = 0.3
damp = 0.1

probe_gamma = probe_mass / damp

T_kin = probe_mass*np.array(vsq_data)/2

plt.plot(T_kin, D_data)

plt.plot(T_kin, T_kin*probe_mu)

In [ ]:
print(settings['run_name'])

In [ ]:
res_x = {}
res_msd = {}
res_vacf = {}

run_name = settings['run_name']
for (i, val) in enumerate(settings['vary_v']):
    prefix = f'{run_name}_{i:05d}_'

    data_x = []
    data_msd = []
    data_vacf = []
    runs = glob(os.path.join(run_name, prefix+'*_processed.h5'))
    print(runs)
    for r in runs:
        with h5py.File(r, 'r') as a:
            data_x.append(a['x'][:])
            data_msd.append(a['msd'][:])
            data_vacf.append(a['vacf'][:])
            r_dt = a['r_dt'][()]
            v_dt = a['v_dt'][()]
    print(data_x[0].shape)
    res_x[val] = np.mean(data_x, axis=0)
    res_msd[val] = np.mean(data_msd, axis=0)
    res_vacf[val] = np.mean(data_vacf, axis=0)
    print(val)


In [ ]:
ls {run_name}

In [ ]:
res_x[5].shape

In [ ]:
probe_mu = np.array(list(map(float, """5 mobility 0.001901381136953548
10 mobility 0.0005232571874781254
15 mobility 0.00022741719358230125
20 mobility 0.00012244118083948134""".split()[2::3])))

In [ ]:
T_kin

In [ ]:
probe_mass